Maybe we define new structure which captures the input parameters specific to each problem in one place (`scale_factor`).
We need to define a range on `d` that is also function dependent (that we could adjust by hand). 

We need to generate three graphs: 
- Histogram of Number of `loc_min` points were found, (so outputs of the Optim routine) and what percentage of them is within a small distance of a critical point of the approximant. As a function of the degree `d` of the approximant.


In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

Can a function export "Optional attributes" like just some constants or its optimal domain of definition? --> works. 

In [ ]:
d_min, d_max = 3, 30
TD = 1.0
# dejong5 parameters (previously from load_function_params)
TR = test_input(dejong5;
    dim=2,
    center=[0.0, 0.0],
    GN=120,
    sample_range=50.0,
    tolerance=3e-4)

@polyvar(x[1:TR.dim]); # Define polynomial ring 

In [ ]:
results = analyze_degrees(TR, x, d_min, d_max, step=1, tol_dist=TD)

In [ ]:
fig_1 = plot_discrete_l2(results, d_min, d_max, 1)
# Commented out save command
# save("discrete_l2.pdf", fig_1)
display(fig_1)

In [ ]:
fig_2 = capture_histogram(results, d_min, d_max, 1, show_legend=false)
# Commented out save command
# save("histogram.pdf", fig_2)
display(fig_2)

In [ ]:
fig_3 = plot_convergence_analysis(results, d_min, d_max, 1)
# Commented out save command
# save("convergence_analysis.pdf", fig_3)
display(fig_3)

In [ ]:
RT = results[20]
df_t = RT.df
df_m = RT.df_min;
sort!(df_t, :z)
df_m_captured = df_m[df_m.captured.==1, :]

In [ ]:
pol_cheb = Constructor(TR, 8, basis=:chebyshev)
inside_mask = points_in_hypercube(df_t, TR)
values_mask = points_in_range(df_t, TR, 24.0)
df_minimizers = df_t[values_mask.&inside_mask, :] # has both `x` (raw) and `y` (optimized)

In [ ]:
fig_4 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_t, df_m_captured, show_captured=true)
# fig_1_p = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_minimizers, df_m, show_captured=false)
# Commented out save command
# save("polyapprox_levelset_min_captured.pdf", fig_4)

In [ ]:
fig_4 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_minimizers, df_m_captured, show_captured=false)

In [ ]:
stats = analyze_converged_points(df_t, TR, results, d_min, d_max, 1)

In [ ]:
fig_5 = plot_distance_statistics(stats)


In [ ]:
stats_2 = analyze_converged_points(df_minimizers, TR, results, d_min, d_max, 1)

Here we had to eliminate the outputs of the Optim routine that were not successful, and the points coming from the approximant. Just because the Optim routine has converged somewhere does not mean it is a good starting point. 

In [ ]:
fig_6 = plot_distance_statistics(stats_2)
# Commented out save command
# save("distance_to_minimizers.pdf", fig_6)